# "Old" Data from the old space that the occupants inhabited

Clayton Miller
Feb 22, 2017

Extract data and analyze data according to Jovan's instructions:

```
First I will start with dates:
7th October till 15th October. According to data looks like 11th and 12th October were weekends although the calendar says 10th and 11th October. 

Sensor readings will have a few adjustments. I will go as they appear in the Grafana visualization tool.

Temperature [deg C] - no corrections
Relative Humidity [%] - no correction
CO [ppm] - no correction
CO2 [ppm] - I need to see you regarding this. I know that Starbuck had garbage reading. Ganzhi numerical values that appear when a cursor is over them are good but visualization is totally off. 
NO2 [ppm] - multiply with 0.378 to get NO2 [micrograms/m3]
Formaldehyde - this was garbage reading. Erase it!
Ozone [ppb] - no corrections
VOC [ppb] - no corrections
Particles [particles/liter] - we will correct this for exclusively PM 2.5 by: number measured x 0.008177 and you will have [micrograms/m3]
IR Wall Temperature - all values should be averaged by adding them all up (8 of them) and just divided by 8. If this is possible would be the best: take that averaged value and add value Clayton_temp.mean and divide them by 2. This should then be Operative Temperature [deg C]. If this is complicated just use an average of 8 values and call that operative temperature.
```

In [108]:
import pandas as pd
from influxdb import DataFrameClient
import pytz
import seaborn as sns
import json
import datetime
import matplotlib.pyplot as plt

from pylab import *
from __future__ import division

In [2]:
%matplotlib inline

In [3]:
host='horsebrokedown-clocktower-1.c.influxdb.com' 
port='8086'
login='influxdb'
pw='a873671b790d4523'
dbname = 'particletest'

client = DataFrameClient(host, port, login, pw, dbname, ssl=True, verify_ssl=True)

In [4]:
seriesname = pd.DataFrame(list(client.query('SHOW SERIES'))[0])

In [5]:
series = seriesname['key'].str.split(",").apply(pd.Series)

In [6]:
co2sensors = series[series[0].str.contains("CO2")]

In [7]:
pointlist = list(series[0])

In [8]:
pointname = pointlist[0]

In [9]:
data = client.query('SELECT * FROM '+pointname+' WHERE time > now() - 1000d')
data = pd.DataFrame(data[pointname])
tz_pytz = pytz.timezone('Asia/Singapore')
data = data.tz_convert(tz_pytz)

In [10]:
#data['pointname'] = data.sensorbank+"_"+data["type"]

In [11]:
data.head()

,sensorbank,type,value
2015-09-04 17:01:18+08:00,CLETUS,CO,4.094721
2015-09-04 17:02:37+08:00,CLETUS,CO,4.094721
2015-09-04 17:03:59+08:00,CLETUS,CO,4.094721
2015-09-04 17:05:18+08:00,CLETUS,CO,4.094721
2015-09-04 17:06:40+08:00,CLETUS,CO,4.094721


In [12]:
data = pd.pivot_table(data, index=data.index, columns=['sensorbank','type'], values='value')

In [13]:
data.head()

sensorbank,CLETUS
type,CO
2015-09-04 17:01:18+08:00,4.094721
2015-09-04 17:02:37+08:00,4.094721
2015-09-04 17:03:59+08:00,4.094721
2015-09-04 17:05:18+08:00,4.094721
2015-09-04 17:06:40+08:00,4.094721


In [14]:
allpoints = pd.DataFrame()
for pointname in pointlist:
    print "Getting "+pointname
    try:
        data = client.query('SELECT * FROM '+pointname+' WHERE time > now() - 1000d')
        data = pd.DataFrame(data[pointname])
        tz_pytz = pytz.timezone('Asia/Singapore')
        data = data.tz_convert(tz_pytz)
        #data['pointname'] = data.sensorbank+"_"+data["type"]
        #data = data.pivot(columns='pointname', values='value')
        data = pd.pivot_table(data, index=data.index, columns=['type','sensorbank'], values='value')
        data = data.resample("10min").mean()
        allpoints = pd.concat([allpoints, data], axis=1)
    #allpoints = pd.merge(allpoints, data, how='outer', right_index=True, left_index=True)
    except:
        print "Point not found"

Getting CLETUS_CO
Getting CLETUS_CO2
Getting CLETUS_FORM
Getting CLETUS_NO2
Getting CLETUS_O3
Getting CLETUS_PART
Getting CLETUS_VOC
Getting CLETUS_hum
Getting CLETUS_temp
Getting CO2_1_CO2_11
Getting CO2_1_CO2_12
Getting CO2_2_CO2_21
Getting CO2_2_CO2_22
Getting CO2_3_CO2_31
Getting CO2_3_CO2_32
Getting Clayton_hum
Getting Clayton_i1
Getting Clayton_i2
Getting Clayton_i3
Getting Clayton_i4
Getting Clayton_i5
Getting Clayton_i6
Getting Clayton_i7
Getting Clayton_i8
Getting Clayton_temp
Getting DALE_CO
Getting DALE_CO2
Getting DALE_FORM
Getting DALE_NO2
Getting DALE_O3
Getting DALE_PART
Getting DALE_VOC
Getting DALE_hum
Getting DALE_temp
Getting ERIC_hum
Getting ERIC_milli
Getting ERIC_temp
Getting GANZHI_CO
Getting GANZHI_CO2
Getting GANZHI_FORM
Getting GANZHI_NO2
Getting GANZHI_O3
Getting GANZHI_PART
Getting GANZHI_VOC
Getting GANZHI_hum
Getting GANZHI_temp
Getting PEACHES_hum
Getting PEACHES_milli
Getting PEACHES_temp
Getting Photon_3_Ball-Hum
Point not found
Getting Photon_3_Ball-Te

In [15]:
#sns.boxplot(allpoints_oct)

In [116]:
allpoints.head()

type                             CO    CO2      FORM       NO2        O3  \
sensorbank                   CLETUS CLETUS    CLETUS    CLETUS    CLETUS   
2015-09-04 16:40:00+08:00       NaN    NaN       NaN       NaN       NaN   
2015-09-04 16:50:00+08:00       NaN    NaN       NaN       NaN       NaN   
2015-09-04 17:00:00+08:00  4.095077    0.0  0.022341  4.083458  4.102058   
2015-09-04 17:10:00+08:00  4.095551    0.0  0.023582  4.083813  4.101820   
2015-09-04 17:20:00+08:00       NaN    NaN       NaN       NaN       NaN   

type                        PART       VOC    hum   temp CO2_11 ...      CO2  \
sensorbank                CLETUS    CLETUS CLETUS CLETUS  CO2_1 ...  TEKANAN   
2015-09-04 16:40:00+08:00    NaN       NaN    NaN    NaN    NaN ...      NaN   
2015-09-04 16:50:00+08:00    NaN       NaN    NaN    NaN    NaN ...      NaN   
2015-09-04 17:00:00+08:00    0.0  0.006053    NaN    NaN    NaN ...      NaN   
2015-09-04 17:10:00+08:00    0.0  0.006231    NaN    NaN    NaN ...      NaN   
2015-09-04 17:20:00+08:00    NaN       NaN    NaN    NaN    NaN ...      NaN   

type                         FORM     NO2      O3    PART     VOC     hum  \
sensorbank                TEKANAN TEKANAN TEKANAN TEKANAN TEKANAN TEKANAN   
2015-09-04 16:40:00+08:00     NaN     NaN     NaN     NaN     NaN     NaN   
2015-09-04 16:50:00+08:00     NaN     NaN     NaN     NaN     NaN     NaN   
2015-09-04 17:00:00+08:00     NaN     NaN     NaN     NaN     NaN     NaN   
2015-09-04 17:10:00+08:00     NaN     NaN     NaN     NaN     NaN     NaN   
2015-09-04 17:20:00+08:00     NaN     NaN     NaN     NaN     NaN     NaN   

type                         temp  VOC       
sensorbank                TEKANAN VOC1 VOC3  
2015-09-04 16:40:00+08:00     NaN  NaN  NaN  
2015-09-04 16:50:00+08:00     NaN  NaN  NaN  
2015-09-04 17:00:00+08:00     NaN  NaN  NaN  
2015-09-04 17:10:00+08:00     NaN  NaN  NaN  
2015-09-04 17:20:00+08:00     NaN  NaN  NaN  

[5 rows x 73 columns]

In [115]:
allpoints.to_csv("./extracted_data/old_space/bulk_data/AllParticleData.csv")

In [16]:
allpoints["CO2"].head()

sensorbank,CLETUS,DALE,GANZHI,STARBUCK,TEKANAN
2015-09-04 16:40:00+08:00,NaN,0.0,NaN,NaN,NaN
2015-09-04 16:50:00+08:00,NaN,0.0,NaN,NaN,NaN
2015-09-04 17:00:00+08:00,0.0,0.0,0.0,0.0,NaN
2015-09-04 17:10:00+08:00,0.0,NaN,0.0,0.0,NaN
2015-09-04 17:20:00+08:00,NaN,NaN,NaN,NaN,NaN
